# Using Generalized Linear Model with Feature Vectors for Event Detections

Finally, we are now using the **feature vectors** described in the [previous notebook](https://github.com/tingsyo/dlgridsat/blob/main/notebook/08_DataFlow_for_Experiments.ipynb) along with the [**Generalized Linear Model**](https://en.wikipedia.org/wiki/Generalized_linear_model) to detect and to predict the events described in the [other notebook](https://github.com/tingsyo/dlgridsat/blob/main/notebook/00_weather_events.ipynb).


## Define the terms

### Weather Events

- **HRD**: Precip >= 40mm/day
- **HRH**: Precip >= 10mm/hr
- **CS**: 寒潮，台北測站24小時內有任一小時10度C以下
- **TYW**: 中央氣象局發布颱風警報
- **NWPTC**: 西北太平洋地區有熱帶氣旋
- **FT**: 中央氣象局地面天氣圖，2000年以後以00Z代表
- **NE**: 彭佳嶼測站日平均風向為東北風(15-75度)及風速達4m/s
- **SWF**: CFSR 850hPa 紅色區域內 u平均>0並且v平均>0並且平均風達3m/s 或者 >6m/s的風速範圍站紅色區域30%


### Feature Vectors

- **PCA** : Principle Component Analysis with 2,048 components.
- **CAE** : Convolutional Auto-Encoder encoded vectors, with a length of 2,048.
- **CVAE**: Variational Auto-Encoder encoded vectors, with a length of 2,048.
- **PTBE**: [ResNet50 pre-trained with Big-Earth dataset](https://tfhub.dev/google/remote_sensing/bigearthnet-resnet50/1), with a length of 2,048.
- **PTIN**: [ResNet50 pre-trained with ImageNet dataset](https://tfhub.dev/tensorflow/resnet_50/feature_vector/1), with a length of 2,048.


## Data Partition

We keep the year 2016 for validation, and 2013~2015 for training and development. For Training and development, we use cross-validation for model tuning.

### Event Dataset
